In [4]:
!pip3 install pandas


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.0 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 499 kB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 13.9 MB 855 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
import json
import pandas as pd

with open("/Users/anarghsidhardhan/Documents/semester2/CS506-DS/Project/new_data/JSON files/Ramps New/Ramps New.json",'r') as f:
    data = json.loads(f.read())
  
df = pd.json_normalize(
    data, 
    record_path =['features']
)

In [8]:
df.head

<bound method NDFrame.head of           type     id geometry.type                      geometry.coordinates  \
0      Feature      0         Point   [-7911843.053800002, 5213855.430799998]   
1      Feature      1         Point   [-7912104.434000002, 5213764.354100004]   
2      Feature      2         Point  [-7912111.2201999985, 5213780.467799999]   
3      Feature      3         Point    [-7912093.786999997, 5213788.37470001]   
4      Feature      4         Point   [-7911853.312899999, 5213875.931500003]   
...        ...    ...           ...                                       ...   
26207  Feature  26207         Point        [-7908859.5842, 5213906.350699998]   
26208  Feature  26208         Point        [-7908620.2472, 5213876.224299997]   
26209  Feature  26209         Point              [-7908621.175, 5213877.0612]   
26210  Feature  26210         Point        [-7908602.0033, 5213865.763899997]   
26211  Feature  26211         Point        [-7910820.3533, 5209827.192599997]  

In [10]:
df.to_csv("./Ramp_score.csv", index=False)
